In [ ]:
import numpy as np
import xarray as xr
from glob import glob
import geopandas as gpd
from utils.functions import sample_from_raster 
from utils.functions import meter2deg
from utils.functions import pixc_geophy_cor
from utils.pixc2raster import pixc2raster
from utils.swot_data_mask import swot_pixc_mask
from utils.swot_data_filter import iter_IQR, pixc_height_local_filtering



In [58]:
lake_name = 'siling_co'
dir_pixc = f'data/{lake_name}-lake/swot-pixc'
path_lake_vec = f'data/{lake_name}-lake/hydrolake_{lake_name}.gpkg'
## Check original .nc file.
paths_pixc = [p for p in glob(dir_pixc + '/SWOT*.nc') if '_masked' not in p and 'filter' not in p and 'height' not in p]
paths_pixc = sorted(paths_pixc)
print(len(paths_pixc))


8


In [59]:
## read vector file of the lake.
lake_gdf = gpd.read_file(path_lake_vec)
lake_gdf


,Hylak_id,Lake_name,Country,Continent,Poly_src,Lake_type,Grand_id,Lake_area,Shore_len,Shore_dev,...,Vol_src,Depth_avg,Dis_avg,Res_time,Elevation,Slope_100,Wshd_area,Pour_long,Pour_lat,geometry
0,147,Siling,China,Asia,SWBD,1,0,1749.53,402.54,2.71,...,1,28.0,0.0,-1.0,4539,-1.0,29077.1,89.047917,31.777083,"MULTIPOLYGON (((88.97853 32.01195, 88.9806 32...."


In [60]:
lake_decrease_gdf = lake_gdf.copy()
lon_center = lake_decrease_gdf.bounds.mean(axis=1).values
utm_zone = np.floor(lon_center/6)+31
epsg_code = f'326{int(utm_zone[0])}'
lake_decrease_gdf = lake_decrease_gdf.to_crs(epsg=epsg_code)
lake_decrease_gdf['geometry'] = lake_decrease_gdf.geometry.buffer(-200)  # interior buffer
lake_decrease_gdf = lake_decrease_gdf.to_crs(epsg=4326)


### data mask

In [61]:
pixcs_masked = []
for i, path in enumerate(paths_pixc):
  print(f'input file: {i} {path}')
  # Define the output path
  path_masked = path.split('.')[0]+'_masked.nc'
  pixc_nc = xr.open_dataset(path, group='pixel_cloud')
  pixc_masked = swot_pixc_mask(pixc_nc=pixc_nc, 
                                vars_sel=['latitude', 'longitude', 'height', 
                                          'solid_earth_tide', 'pole_tide', 
                                          'load_tide_fes', 'iono_cor_gim_ka', 'geoid',
                                          ],
                                region_gdf=lake_decrease_gdf, 
                                path_masked=path_masked)
  pixcs_masked.append(pixc_masked)


input file: 0 data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_015_217_210L_20240516T015222_20240516T015234_PIC0_01.nc
file written to data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_015_217_210L_20240516T015222_20240516T015234_PIC0_01_masked.nc
input file: 1 data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_016_217_210L_20240605T223729_20240605T223740_PIC0_01.nc
file written to data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_016_217_210L_20240605T223729_20240605T223740_PIC0_01_masked.nc
input file: 2 data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_017_217_210L_20240626T192231_20240626T192243_PIC0_01.nc
file written to data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_017_217_210L_20240626T192231_20240626T192243_PIC0_01_masked.nc
input file: 3 data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_018_217_210L_20240717T160736_20240717T160747_PIC0_01.nc
file written to data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_018_217_210L_20240717T160736_20240717T160747_PIC0_01_masked.nc
input file: 4 data/siling_co-lake/sw

### data filter 

In [62]:
pixcs_filtered_ds = []
for i, pixc_masked in enumerate(pixcs_masked):
    pixc_ht_filter_ds = pixc_masked[['geoid', 'height']]
    ## 1. geophysical correction for height, and convert to orthometric height
    pixc_ht_cor = pixc_geophy_cor(pixc_nc=pixc_masked)
    pixc_ht_ortho = pixc_ht_cor - pixc_masked.geoid.values
    pixc_ht_filter_ds = pixc_ht_filter_ds.assign({'ht_ortho': (("points",), pixc_ht_ortho)})
    pixc_ht_filter_ds.ht_ortho.attrs['description'] = 'PIXC height data with geophysical correction and orthometric correction (use geoid variable)'

    ## 2. height filtering 
    ## 2.1 height filtering in global region using IQR method
    pixc_ht_ortho_filter1, IQR = iter_IQR(pixc_ht_ortho, IQR_thre=0.3, iter_max=5)
    pixc_ht_ortho_filter1 = pixc_ht_ortho_filter1.filled(np.nan)
    pixc_ht_filter_ds = pixc_ht_filter_ds.assign({'ht_ortho_filter1': (("points",), pixc_ht_ortho_filter1)})
    pixc_ht_filter_ds.ht_ortho_filter1.attrs['description'] = 'PIXC height data with global filtering using IQR method'
    ## 2.2 height filtering in local region
    pixc_ht_ortho_filter2 = pixc_height_local_filtering(pixc_height=pixc_ht_ortho_filter1, 
                                              pixc_lonlat=(pixc_ht_filter_ds.longitude.values, pixc_ht_filter_ds.latitude.values), 
                                              thre=0.2, 
                                              radius_m=500)
    pixc_ht_filter_ds = pixc_ht_filter_ds.assign({'ht_ortho_filter2': (("points",), pixc_ht_ortho_filter2)})
    pixc_ht_filter_ds.ht_ortho_filter2.attrs['description'] = 'PIXC height with both global and local filtering'
    pixc_ht_filter_ds = pixc_ht_filter_ds.assign({'ht_valid_ids': (("points",), ~np.isnan(pixc_ht_ortho_filter2))})
    pixc_ht_filter_ds.ht_valid_ids.attrs['description'] = 'PIXC height indices of valid data'
    pixcs_filtered_ds.append(pixc_ht_filter_ds)

    # # 3. save the filtered data
    path_pixc_filtered = paths_pixc[i].split('.')[0]+'_masked_filtered.nc'
    pixc_ht_filter_ds.to_netcdf(path_pixc_filtered, mode='w', format='NETCDF4')  ## save the filtered data
    print(f'Filtered data saved to: {path_pixc_filtered}')



Filtered data saved to: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_015_217_210L_20240516T015222_20240516T015234_PIC0_01_masked_filtered.nc
Filtered data saved to: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_016_217_210L_20240605T223729_20240605T223740_PIC0_01_masked_filtered.nc
Filtered data saved to: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_017_217_210L_20240626T192231_20240626T192243_PIC0_01_masked_filtered.nc
Filtered data saved to: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_018_217_210L_20240717T160736_20240717T160747_PIC0_01_masked_filtered.nc
Filtered data saved to: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_019_217_210L_20240807T125240_20240807T125252_PIC0_01_masked_filtered.nc
Filtered data saved to: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_020_217_210L_20240828T093745_20240828T093756_PIC0_01_masked_filtered.nc
Filtered data saved to: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_021_217_210L_20240918T062251_20240918T062302_PIC0_01_masked_filtered.nc
Filter

### calculate the height heterogeneity

In [63]:
## calculate the height heterogeneity
xmin, ymin, xmax, ymax = lake_gdf.geometry[0].buffer(0.01).bounds
raster_extent = (xmin, xmax, ymin, ymax)
lat_center = pixcs_filtered_ds[0]['geoid'].latitude.mean().values
res_lon, res_lat = meter2deg(meter=500, lat=lat_center)

## 1. calculate the spatial heterogeneity of height anomalies
rasters_ht_spahet = []

for pixc_filtered_ds in pixcs_filtered_ds:
    pixc_geoid_cor_ds = pixc_filtered_ds[['geoid', 'ht_valid_ids']]
    ## calculate corrected geoid
    pixc_ht_ellip = pixc_filtered_ds['ht_ortho_filter2'].values + pixc_filtered_ds['geoid'].values
    pixc_ht_ellip_spavar = pixc_ht_ellip - np.nanmean(pixc_ht_ellip) ## i.e., spatial variance of height anomalies
    geoid_mean = np.nanmean(pixc_filtered_ds['geoid'])
    pixc_geoid_cor = geoid_mean + pixc_ht_ellip_spavar    ## Pixc Geoid correction

    ## 2. convert pixc to raster
    # for geoid_cor:
    raster_geoid_cor = pixc2raster(pixc_var = pixc_ht_ellip_spavar, 
                        raster_extent=raster_extent,
                        pixc_lonlat=(pixc_geoid_cor_ds.longitude.values, pixc_geoid_cor_ds.latitude.values), 
                        resolution=(res_lon, res_lat))
    raster_geoid_cor.attrs = pixc_geoid_cor_ds.attrs
    rasters_ht_spahet.append(raster_geoid_cor)
    ## for geoid
    raster_geoid = pixc2raster(pixc_var = pixc_geoid_cor, 
                        raster_extent=raster_extent,
                        pixc_lonlat=(pixc_geoid_cor_ds['geoid'].longitude.values, pixc_geoid_cor_ds['geoid'].latitude.values), 
                        resolution=(res_lon, res_lat))
    ## for geoid_spahet
    raster_geoid_spahet = pixc2raster(pixc_var = pixc_geoid_cor-np.nanmean(pixc_geoid_cor_ds['geoid'].values), 
                        raster_extent=raster_extent,
                        pixc_lonlat=(pixc_geoid_cor_ds['geoid'].longitude.values, pixc_geoid_cor_ds['geoid'].latitude.values), 
                        resolution=(res_lon, res_lat))


## 2. temporal smoothing using median filter
rasters_ht_spahet_da = xr.concat(rasters_ht_spahet, dim='date') 
raster_ht_spahet_smooth = rasters_ht_spahet_da.median(dim='date', keep_attrs=True)  # temporal smoothing
raster_ht_spahet_smooth.name = "height_spahet_smoothed"

## 3. save to NetCDF
raster_geoid_spahet.attrs['description'] = 'the original spatial heterogeneity of height anomalies of the lake, '
raster_ht_spahet_smooth.attrs['description'] = 'the swot data-derived spatial heterogeneity of height anomalies of the lake'
raster_height_spahet_ds = xr.Dataset({
    'geoid': raster_geoid.rename('geoid'),
    'geoid_spahet': raster_geoid_spahet.rename('geoid_spahet'),
    'ht_spahet_smoothed': raster_ht_spahet_smooth.rename('ht_spahet_smoothed')
})
raster_height_spahet_ds
# Save as NetCDF file
path_raster_height_spahet = dir_pixc + '/raster_height_spahet.nc'
raster_height_spahet_ds.to_netcdf(path_raster_height_spahet)
print(f'data saved to: {path_raster_height_spahet}')


data saved to: data/siling_co-lake/swot-pixc/raster_height_spahet.nc


### calculate pixc-based height_ortho and corrected height_ortho 

In [64]:
for i, pixc_filtered in enumerate(pixcs_filtered_ds):
    pixc_ht_ds = pixc_filtered[['geoid', 'ht_ortho_filter2']]
    ## 1. extract the pixc corrected geoid from the raster data 
    pixc_ht_spahet = sample_from_raster(
        raster_value = raster_height_spahet_ds.ht_spahet_smoothed.values,
        raster_x = raster_height_spahet_ds.x.values,
        raster_y = raster_height_spahet_ds.y.values,
        points_x = pixc_filtered.longitude.values,
        points_y = pixc_filtered.latitude.values
    )
    pixc_ht_ds = pixc_ht_ds.assign({'ht_spahet': (("points",), pixc_ht_spahet)})
    pixc_ht_ds['ht_spahet'].attrs['description'] = 'spatial heterogeneity of lake height'
    ## 2. calculate pixc height with corrected geoid
    geoid_cor = pixc_filtered.geoid.mean(dim='points').values + pixc_ht_ds['ht_spahet'].values
    pixc_ht_ortho2 = pixc_filtered.height.values - geoid_cor
    pixc_ht_ortho2[~pixc_filtered['ht_valid_ids'].values] = np.nan   ## mask invalid values
    pixc_ht_ds = pixc_ht_ds.assign({'ht_ortho_cor_filter2': (("points",), pixc_ht_ortho2)})
    pixc_ht_ds['ht_ortho_cor_filter2'].attrs['description'] = 'orthometric height using corrected geoid'
    ### 3. save the pixc heights to a new NetCDF file
    path_pixc_height = paths_pixc[i].replace('.nc', '_height.nc')
    pixc_ht_ds.to_netcdf(path_pixc_height)
    print(f"Path to save: {path_pixc_height}")



Path to save: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_015_217_210L_20240516T015222_20240516T015234_PIC0_01_height.nc
Path to save: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_016_217_210L_20240605T223729_20240605T223740_PIC0_01_height.nc
Path to save: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_017_217_210L_20240626T192231_20240626T192243_PIC0_01_height.nc
Path to save: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_018_217_210L_20240717T160736_20240717T160747_PIC0_01_height.nc
Path to save: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_019_217_210L_20240807T125240_20240807T125252_PIC0_01_height.nc
Path to save: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_020_217_210L_20240828T093745_20240828T093756_PIC0_01_height.nc
Path to save: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_021_217_210L_20240918T062251_20240918T062302_PIC0_01_height.nc
Path to save: data/siling_co-lake/swot-pixc/SWOT_L2_HR_PIXC_022_217_210L_20241009T030758_20241009T030809_PIC0_01_height.nc
